# Решение задачи "Классификация болезней яблонь" RuCode-3

В этом ноутбуке будем решать задачу классификации болезней яблонь с помощью предобученной нейронной сети ResNet-18.

### Скачиваем данные на Google Colab и распаковываем:

In [ ]:
! wget "https://s98vla.storage.yandex.net/rdisk/a185914f68628cd93842d4a6a70dcb0d62a16e7e1106bf3c4cd28e1f12c1795e/606b4686/aZ8Akc9CqtcaAMvg9MkExSJefHHGZZYdTl2AU-VqM4afERxsBLTpt8qwBoFEHzTPyT91bTsmoQx6cwIsgiEZFg==?uid=316082386&filename=train.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=316082386&fsize=5901567&hid=539d9e8e45b5325472ea6667b22be86c&media_type=compressed&tknv=v2&etag=e20d5f1607aab3615d9c6dab65c2c61c&rtoken=k4PH1F4bj22Z&force_default=yes&ycrid=na-fea6fc7ef38a35b2c39f5ebc2fe4bc33-downloader5h&ts=5bf3ce4da2d80&s=26c22f6add2ee51ad5e848554e5813842dda62c96e5c279c59069a9102b66992&pb=U2FsdGVkX1_yuow0K5VWKBzcLUifWeCtPIK432oYHEqPxtzC1RqtiodajzZXnCdvCMGe7FZ-BPpY4_9z8aqI1QW47a_-pezgoKSHUVTsePA" -O train.zip
! wget "https://s147vla.storage.yandex.net/rdisk/b735f673d07382bc37245810c24d803ec6388172b39cfa2b7d092b18b4a6746d/606b46a8/aZ8Akc9CqtcaAMvg9MkExdtkgBWnfCSxc1Dv7y1NU9c9_o0JWRHuxfnP2i148GqPcl3BUbVgs-Mvl1Yht-sqww==?uid=316082386&filename=test_classes.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=316082386&fsize=1341245&hid=ac6712119a1a70fa56686892aa7c81df&media_type=compressed&tknv=v2&etag=22a39c0e331780d6f98d164944612e71&rtoken=eXZekDnBFGGd&force_default=yes&ycrid=na-42fa07d4806adce4d5247aabeb3475a3-downloader5h&ts=5bf3ce6e0fa00&s=6d922208f0d9be353f461565faf3c754e27f77ebdff51d22db0aba01d1a01c98&pb=U2FsdGVkX1_8XZNfYSLPzG0xNmB9O-HQpEf66BsuCdvOR0zUYzcpwqGw3bPCrsY0jrepN-0mHN2_0F8t-FcQMR7OvDIH55r2sHwAfe1PKR0" -O test_classes.zip

--2021-04-05 15:01:11--  https://s98vla.storage.yandex.net/rdisk/a185914f68628cd93842d4a6a70dcb0d62a16e7e1106bf3c4cd28e1f12c1795e/606b4686/aZ8Akc9CqtcaAMvg9MkExSJefHHGZZYdTl2AU-VqM4afERxsBLTpt8qwBoFEHzTPyT91bTsmoQx6cwIsgiEZFg==?uid=316082386&filename=train.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=316082386&fsize=5901567&hid=539d9e8e45b5325472ea6667b22be86c&media_type=compressed&tknv=v2&etag=e20d5f1607aab3615d9c6dab65c2c61c&rtoken=k4PH1F4bj22Z&force_default=yes&ycrid=na-fea6fc7ef38a35b2c39f5ebc2fe4bc33-downloader5h&ts=5bf3ce4da2d80&s=26c22f6add2ee51ad5e848554e5813842dda62c96e5c279c59069a9102b66992&pb=U2FsdGVkX1_yuow0K5VWKBzcLUifWeCtPIK432oYHEqPxtzC1RqtiodajzZXnCdvCMGe7FZ-BPpY4_9z8aqI1QW47a_-pezgoKSHUVTsePA
Resolving s98vla.storage.yandex.net (s98vla.storage.yandex.net)... 93.158.162.227, 2a02:6b8:c0e:182:0:41af:3c42:3478
Connecting to s98vla.storage.yandex.net (s98vla.storage.yandex.net)|93.158.162.227|:443... connected.
HTTP request sent, awaiti

In [ ]:
! unzip train.zip
! unzip test_classes.zip

In [ ]:
! ls test_classes/

0_гниль  1_ржавчина  2_парша


### Строим нейросеть

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

Создаем даталоадеры для обучения нейросети:

In [ ]:
images_dataset = torchvision.datasets.ImageFolder("./train", transform=transforms.Compose([                                                                  
  transforms.ToTensor(),
  # нормализуем как в ImageNet
  torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                   [0.229, 0.224, 0.225]),
                                                                                                    
]))

images_dataloader = torch.utils.data.DataLoader(images_dataset, batch_size=16,
                                          shuffle=True)

Будем использовать предобученную на ImageNet сеть ResNet18:

In [ ]:
net = models.resnet18(True, True).cuda() 
net  

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Заменим последний полносвязный слой сети на слой, который будет выдавать 3 значения на выходе (т.к. у нас 3 класса):

In [ ]:
net.fc = nn.Linear(512, 3)

Заморозим все слом нейросети, кроме самого последнего, только что добавленного fc-слоя. Будем обучать только последний слой сети.

In [ ]:
for i, child in enumerate(net.children()):
    if i == 9: 
        break
    for param in child.parameters():
        param.requires_grad = False  

Объявляем лосс-функцию и оптимизатор:

In [ ]:
# стандартная лосс-функция для задачи классификации
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# для обучения на GPU
device = 'cuda:0'
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Обучаем сеть 5 эпох:

In [ ]:
n_epochs = 5
print_every = 10

total_step = len(images_dataloader)

for epoch in range(1, n_epochs+1):

    print(f'Epoch {epoch}\n')
    for batch_idx, (data, target) in enumerate(images_dataloader):
        # кладем данные на GPU
        data, target = data.to(device), target.to(device)

        # делаем шаг обучения сети
        optimizer.zero_grad()
        outputs = net(data)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        if (batch_idx) % 20 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))

Epoch 1

Epoch [1/5], Step [0/25], Loss: 1.1735
Epoch [1/5], Step [20/25], Loss: 0.7808
Epoch 2

Epoch [2/5], Step [0/25], Loss: 0.6210
Epoch [2/5], Step [20/25], Loss: 0.3811
Epoch 3

Epoch [3/5], Step [0/25], Loss: 0.3694
Epoch [3/5], Step [20/25], Loss: 0.2428
Epoch 4

Epoch [4/5], Step [0/25], Loss: 0.2725
Epoch [4/5], Step [20/25], Loss: 0.1689
Epoch 5

Epoch [5/5], Step [0/25], Loss: 0.1234
Epoch [5/5], Step [20/25], Loss: 0.2875


### Тестируем обученную нейросеть на тестовом наборе картинок:

In [ ]:
images_testset = torchvision.datasets.ImageFolder("./test_classes", transform=transforms.Compose([                                                                  
  transforms.ToTensor(),
  torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                   [0.229, 0.224, 0.225]),
                                                                                                    
]))

images_testloader = torch.utils.data.DataLoader(images_testset, batch_size=1,
                                          shuffle=False)

In [ ]:
batch_loss = 0
total=0
correct=0

with torch.no_grad():
        net.eval()

        for data, target in (images_testloader):
            # кладем данные на GPU
            data, target = data.to(device), target.to(device)
            outputs = net(data)
            # считаем loss
            loss = criterion(outputs, target)
            batch_loss += loss.item()

            # считаем accuracy
            _, pred = torch.max(outputs, dim=1)
            correct += torch.sum(pred==target).item()
            total += target.size(0)

        print("Acc", 100 * correct/total)
        print("Loss", batch_loss/len(images_testloader))

Acc 100.0
Loss 0.1592358947162413


Вуаля!